In [2]:
import numpy as np
import random

import torch
import torch.nn as nn
import torch.optim as optim

from torchvision import models

from tqdm import tqdm
import os
os.environ["KMP_DUPLICATE_LIB_OK"]  =  "TRUE"
from transfer_recognition import ImageTransform , make_datapath_list , HymenopteraDataset

In [3]:

train_list = make_datapath_list(phase="train")
val_list = make_datapath_list(phase="val")

size = 224
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)
train_dataset = HymenopteraDataset(
    file_list=train_list, transform=ImageTransform(size, mean, std), phase='train')

val_dataset = HymenopteraDataset(
    file_list=val_list, transform=ImageTransform(size, mean, std), phase='val')

batch_size = 8

train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True)

val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False)

dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}

./data/hymenoptera_data/train/**/*.jpg
./data/hymenoptera_data/val/**/*.jpg


In [4]:

use_pretrained = True  
net = models.vgg16(pretrained=use_pretrained)
net.classifier[6] = nn.Linear(in_features=4096, out_features=2)
net.train()
criterion = nn.CrossEntropyLoss()

In [5]:
# 微调中需要学习的参数保存到下面三个列表中

params_to_update_1 = []
params_to_update_2 = []
params_to_update_3 = []

# 需要学习的网络层
update_param_names_1 = ["features"]
update_param_names_2 = ["classifier.0.weight",
                        "classifier.0.bias", "classifier.3.weight", "classifier.3.bias"]
update_param_names_3 = ["classifier.6.weight", "classifier.6.bias"]

# 打开对应网络层的梯度计算
for name, param in net.named_parameters():
    if update_param_names_1[0] in name:
        param.requires_grad = True
        params_to_update_1.append(param)
        print("params_to_update_1保存：", name)

    elif name in update_param_names_2:
        param.requires_grad = True
        params_to_update_2.append(param)
        print("params_to_update_2保存：", name)

    elif name in update_param_names_3:
        param.requires_grad = True
        params_to_update_3.append(param)
        print("params_to_update_3保存：", name)

    else:
        param.requires_grad = False
        print("不进行计算梯度，我们不学习这层", name)
# 微调中需要学习的参数保存到下面三个列表中

params_to_update_1 = []
params_to_update_2 = []
params_to_update_3 = []

# 需要学习的网络层
update_param_names_1 = ["features"]
update_param_names_2 = ["classifier.0.weight",
                        "classifier.0.bias", "classifier.3.weight", "classifier.3.bias"]
update_param_names_3 = ["classifier.6.weight", "classifier.6.bias"]

# 打开对应网络层的梯度计算
for name, param in net.named_parameters():
    if update_param_names_1[0] in name:
        param.requires_grad = True
        params_to_update_1.append(param)
        print("params_to_update_1保存：", name)

    elif name in update_param_names_2:
        param.requires_grad = True
        params_to_update_2.append(param)
        print("params_to_update_2保存：", name)

    elif name in update_param_names_3:
        param.requires_grad = True
        params_to_update_3.append(param)
        print("params_to_update_3保存：", name)

    else:
        param.requires_grad = False
        print("不进行计算梯度，我们不学习这层", name)


params_to_update_1保存： features.0.weight
params_to_update_1保存： features.0.bias
params_to_update_1保存： features.2.weight
params_to_update_1保存： features.2.bias
params_to_update_1保存： features.5.weight
params_to_update_1保存： features.5.bias
params_to_update_1保存： features.7.weight
params_to_update_1保存： features.7.bias
params_to_update_1保存： features.10.weight
params_to_update_1保存： features.10.bias
params_to_update_1保存： features.12.weight
params_to_update_1保存： features.12.bias
params_to_update_1保存： features.14.weight
params_to_update_1保存： features.14.bias
params_to_update_1保存： features.17.weight
params_to_update_1保存： features.17.bias
params_to_update_1保存： features.19.weight
params_to_update_1保存： features.19.bias
params_to_update_1保存： features.21.weight
params_to_update_1保存： features.21.bias
params_to_update_1保存： features.24.weight
params_to_update_1保存： features.24.bias
params_to_update_1保存： features.26.weight
params_to_update_1保存： features.26.bias
params_to_update_1保存： features.28.weight
params_

In [6]:
optimizer = optim.SGD([
    {'params': params_to_update_1, 'lr': 1e-4},
    {'params': params_to_update_2, 'lr': 5e-4},
    {'params': params_to_update_3, 'lr': 1e-3}
], momentum=0.9)

In [7]:
def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):

    
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用的设备是：", device)

   
    net.to(device)  
    torch.backends.cudnn.benchmark = True


    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-------------')

        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()  
            else:
                net.eval()   

            epoch_loss = 0.0  
            epoch_corrects = 0 

         
            if (epoch == 0) and (phase == 'train'):
                continue

        
            for inputs, labels in tqdm(dataloaders_dict[phase]):

          
                inputs = inputs.to(device)
                labels = labels.to(device)

               
                optimizer.zero_grad()

               
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = net(inputs)
                    loss = criterion(outputs, labels)  
                    _, preds = torch.max(outputs, 1)

  
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

      
                    epoch_loss += loss.item() * inputs.size(0)  
  
                    epoch_corrects += torch.sum(preds == labels.data)

     
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double(
            ) / len(dataloaders_dict[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))


In [8]:
torch.cuda.empty_cache()

In [9]:
num_epochs=2
train_model(net, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)

使用的设备是： cuda:0
Epoch 1/2
-------------


100%|██████████| 20/20 [00:06<00:00,  3.14it/s]


val Loss: 0.6745 Acc: 0.5752
Epoch 2/2
-------------


100%|██████████| 31/31 [00:11<00:00,  2.77it/s]


train Loss: 0.2741 Acc: 0.8683


100%|██████████| 20/20 [00:02<00:00,  7.53it/s]

val Loss: 0.1057 Acc: 0.9608
